In [32]:
import numpy as np
import pandas as pd
import os

map_name = 'HRL'

# Load MPC data
mpc_csv_file_name = 'mpc_data_1_4.csv'
mpc_csv_file_path = os.path.join(map_name, mpc_csv_file_name)
mpc_csv = pd.read_csv(mpc_csv_file_path)
print(mpc_csv)


      Iteration       s_cur     w_cur    mu_cur         v         angle  \
0             2   74.217653  0.335779 -0.752138  2.000000  0.000000e+00   
1             3   74.233920  0.294345 -0.752138  1.500000 -3.000000e-01   
2             4   74.283083  0.249805 -0.664348  1.500000  1.214921e-14   
3             5   74.316007  0.204357 -0.674317  1.500000  3.000000e-01   
4             6   74.374406  0.158889 -0.615090  1.500000  4.438701e-01   
...         ...         ...       ...       ...       ...           ...   
1494       1496  121.127939 -0.011893 -0.005647  2.792982  2.007215e-02   
1495       1497  121.261184 -0.012330 -0.005487  2.806648  1.768389e-02   
1496       1498  121.403033 -0.013668 -0.011613  2.819335  2.985897e-02   
1497       1499  121.534758 -0.014316 -0.011966  2.832333  2.629047e-02   
1498       1500  121.675266 -0.015975 -0.019116  2.844820  4.132759e-02   

      Racecar X  Racecar Y   Racecar heading  
0     31.499923  56.500000          0.000000  
1    

In [33]:
# Transfer mpc x,y,vx to numpy array
mpc_data = mpc_csv[['Racecar X', 'Racecar Y', 'v']].to_numpy()
print(mpc_data)

[[31.4999226  56.5         2.        ]
 [31.55959619 56.5         1.5       ]
 [31.62751808 56.5         1.5       ]
 ...
 [75.04736877 49.55986997  2.8193352 ]
 [75.07581616 49.42486923  2.83233316]
 [75.1052515  49.28939125  2.84481966]]


In [34]:
# Load centerline
centerline_file_name = f'{map_name}_centerline.csv'
centerline_file_path = os.path.join(map_name, centerline_file_name)
centerline_csv = pd.read_csv(centerline_file_path)

# Transfer centerline to numpy
centerline_data = centerline_csv[['x','y']].to_numpy()
print(centerline_data)

[[1449.  199.]
 [1448.  200.]
 [1447.  200.]
 ...
 [1452.  199.]
 [1451.  199.]
 [1450.  199.]]


In [35]:
# Load map resolution
map_config_path = os.path.join(map_name, f'{map_name}_map.yaml')

import yaml
with open(map_config_path, 'r') as file:
    map_config_dict = yaml.safe_load(file)
print(map_config_dict)

map_resolution = map_config_dict['resolution']

{'image': 'HRL_map.png', 'negate': 0, 'occupied_thresh': 0.75, 'free_thresh': 0.25, 'resolution': 0.05, 'origin': [0.0, 0.0, 0.0], 'starting_pose': [31.4, 56.5, 0.0], 'starting_index': 1440}


In [36]:
# Transfer centerline data into resolution
centerline_data = centerline_data*map_resolution

In [37]:
# Cut the start and end of mpc data 
print(mpc_data[0], mpc_data.shape[0])
mpc_data_clipped = mpc_data[25:-25]
print(mpc_data_clipped[0], mpc_data_clipped.shape[0])

[31.4999226 56.5        2.       ] 1499
[33.42327733 56.67524129  1.96987792] 1449


In [51]:
# Find nearest point in centerline for each point in mpc data and add velocity 
centerline_data_vx = np.zeros(shape=(centerline_data.shape[0],3))

mpc_data_xy = mpc_data_clipped[:, :2]

# Finde für jeden Punkt in array1 den Index des nächsten Punktes in array2
nearest_indices = []

for point in centerline_data:
    # Berechne die quadratische Distanz zu allen Punkten in array2
    distances = np.sum((mpc_data_xy - point) ** 2, axis=1)
    # Finde den Index des minimalen Werts (nächster Punkt)
    nearest_index = np.argmin(distances)
    nearest_indices.append(nearest_index)

# Ausgabe der Indizes der nächstgelegenen Punkte
print(nearest_indices)

for i in range(centerline_data_vx.shape[0]):
    centerline_data_vx[i] = np.array([centerline_data[i][0], centerline_data[i][1], mpc_data_clipped[nearest_indices[i]][2]])
    print(centerline_data_vx[i])

[np.int64(594), np.int64(595), np.int64(595), np.int64(596), np.int64(596), np.int64(597), np.int64(597), np.int64(598), np.int64(598), np.int64(599), np.int64(599), np.int64(600), np.int64(601), np.int64(601), np.int64(602), np.int64(602), np.int64(603), np.int64(603), np.int64(604), np.int64(604), np.int64(605), np.int64(605), np.int64(606), np.int64(606), np.int64(607), np.int64(607), np.int64(608), np.int64(608), np.int64(609), np.int64(609), np.int64(610), np.int64(610), np.int64(611), np.int64(611), np.int64(612), np.int64(613), np.int64(613), np.int64(614), np.int64(614), np.int64(615), np.int64(615), np.int64(616), np.int64(616), np.int64(617), np.int64(617), np.int64(618), np.int64(618), np.int64(619), np.int64(619), np.int64(620), np.int64(620), np.int64(621), np.int64(621), np.int64(621), np.int64(622), np.int64(622), np.int64(623), np.int64(623), np.int64(624), np.int64(624), np.int64(625), np.int64(625), np.int64(626), np.int64(626), np.int64(626), np.int64(627), np.int64(

In [55]:
# save as csv 
output_file_name = f'{map_name}_centerline_mpc.csv'
centerline_mpc = pd.DataFrame(centerline_data_vx, columns=['x', 'y', 'v'])
centerline_mpc.to_csv(os.path.join(map_name, output_file_name), index=False)